## U-net implementation with Tensorflow
### Matty Vermet

In [ ]:
import os
import sys
import random
import warnings
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPool2D,Concatenate,Add,Dropout,ReLU,Conv2DTranspose,UpSampling2D

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
# Initialize parameters
BATCH_SIZE = 30 
IMG_WIDTH = 224 
IMG_HEIGHT = 224 
IMG_CHANNELS = 3
# wrong = 0
# threshold = 1 - 0.477
#threshold_PCA = 54.0
threshold_samples = 0.5

# Paths to images and masks 
TRAIN_PATH_IMG_1 = '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images'
TRAIN_PATH_MASK_1 = '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/masks'
VAL_PATH_IMG_2 = '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images'
VAL_PATH_MASK_2 = '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/masks'
TEST_PATH_IMG_3 = '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images'
TEST_PATH_MASK_3 = '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/masks'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42

ids_1 = next(os.walk(TRAIN_PATH_IMG_1))[2]
ids_2 = next(os.walk(VAL_PATH_IMG_2))[2]
ids_3 = next(os.walk(TEST_PATH_IMG_3))[2]

np.random.seed(10)

### Remove outliers


In [ ]:
# Remove outliers from train, validation, test set
noise_train = ['../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_584.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_586.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_604.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_748.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_750.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_780.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_811.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_812.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_813.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_828.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_830.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_832.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_833.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_996.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_998.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1147.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1148.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1149.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1152.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1155.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1158.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1160.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1161.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1164.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1166.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1432.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1433.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1512.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1578.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1614.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1615.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1616.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1617.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1618.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1619.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1620.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1629.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1632.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1704.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1705.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1707.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1708.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1709.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1723.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1724.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1725.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1748.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1749.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1750.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1751.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1752.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1753.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1859.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1864.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1870.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1880.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1923.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1939.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1940.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1945.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1946.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1966.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1967.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1968.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1969.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1970.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1971.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1972.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1973.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1974.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1975.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1976.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1977.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1978.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1979.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2007.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2009.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2019.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2020.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2022.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2098.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2108.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2109.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2110.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2111.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2115.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2131.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2132.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2133.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2134.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2135.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2137.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2163.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2164.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2165.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2174.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2176.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2202.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2263.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2264.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2265.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2267.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2406.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2407.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2462.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2463.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2464.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2465.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2515.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2550.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2551.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2552.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2626.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2636.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2639.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2640.npy']

noise_val =  ['../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_544.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_679.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_680.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_724.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_749.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_750.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_752.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_753.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1028.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1029.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1241.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1248.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1249.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1403.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1404.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1434.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1435.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1436.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1440.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1470.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1471.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1472.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1473.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1474.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1475.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1476.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1477.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1478.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1524.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1526.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1538.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1539.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1540.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1541.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1542.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1543.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1544.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1545.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1546.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1558.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1559.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1560.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1600.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1601.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1607.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1651.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1653.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1657.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1660.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1661.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1662.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1665.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1684.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1685.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1686.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1687.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1688.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1689.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1690.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1691.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1692.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1693.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1694.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1695.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1696.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1697.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1698.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1699.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1700.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1701.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1702.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1738.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1741.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1742.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1743.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1746.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1749.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1845.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1846.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1847.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1848.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1849.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1850.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1851.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1852.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1853.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1876.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1889.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1921.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1928.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1929.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1930.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1931.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1932.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1933.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1936.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1998.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1999.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2000.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2003.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2004.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2005.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2019.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2021.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2268.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2269.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2310.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2315.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2397.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2450.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2494.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2508.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2511.npy']

noise_test = ['../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_236.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_546.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_735.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_754.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_762.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_778.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_780.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_784.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1016.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1076.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1078.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1079.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1083.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1085.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1088.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1307.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1438.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1506.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1507.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1508.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1509.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1510.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1511.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1518.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1519.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1523.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1566.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1567.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1568.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1569.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1570.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1571.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1572.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1611.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1635.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1636.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1645.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1646.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1647.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1648.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1664.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1665.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1666.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1667.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1668.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1669.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1670.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1743.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1757.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1777.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1779.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1780.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1797.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1803.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1804.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1805.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1832.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1833.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1834.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1835.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1836.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1837.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1838.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1839.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1840.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1841.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1842.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1843.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1844.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1845.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1846.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1847.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1848.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1849.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1850.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1851.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1852.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1853.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1854.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1855.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1856.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1857.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1858.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1859.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1860.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1861.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1862.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1894.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1896.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1897.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1987.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1988.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1989.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1990.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1991.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1992.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2023.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2025.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2026.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2063.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2064.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2065.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2066.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2067.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2074.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2123.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2124.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2138.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2142.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2143.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2373.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2375.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2694.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2695.npy']

In [ ]:
outliers_train_PCA = ['../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1061.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1726.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1727.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1728.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1729.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1730.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1731.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1751.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1753.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2212.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2267.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2329.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2331.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2377.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2378.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2421.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2422.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2423.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2424.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2425.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2426.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2427.npy']

outliers_val_PCA = ['../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1420.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1548.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1549.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1550.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1552.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1553.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1558.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1572.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1573.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1889.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2064.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2068.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2069.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2167.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2169.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2171.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2172.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2173.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2175.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2224.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2258.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2260.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2261.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2262.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2263.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2264.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2265.npy']

outliers_test_PCA = ['../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1007.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1611.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1612.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1649.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1650.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1651.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1668.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1670.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1700.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2187.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2188.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2317.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2347.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2383.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2384.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2387.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2548.npy']

In [ ]:
outliers_train = ['../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_984.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1487.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1513.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1754.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1760.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2305.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2308.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2309.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2310.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2311.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2312.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2414.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2435.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2436.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2439.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2440.npy',
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2651.npy']

outliers_val = ['../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_492.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1326.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1969.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2150.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2151.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2273.npy',
 '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_2276.npy']

outliers_test = ['../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_866.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1355.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1673.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1783.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2098.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2308.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2309.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2310.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2311.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2332.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2392.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2394.npy',
 '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_2630.npy']

common_train = ['../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1753.npy', 
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_1751.npy', 
 '../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/images/image_2267.npy']

common_val = ['../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1558.npy',
    '../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/images/image_1889.npy']

common_test = ['../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1668.npy', 
    '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1670.npy', 
    '../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/images/image_1611.npy']

# when noise and PCA are use > remove overlap from noise
noise_train = list(set(noise_train) - set(common_train))
noise_val = list(set(noise_val) - set(common_val))
noise_test = list(set(noise_test) - set(common_test))

### Load the training data (Fold 1)

In [ ]:
# Load train images and corresponding masks from Fold 1
removal = outliers_train + noise_train + outliers_train_PCA

# Create empty arrays to store images and masks
X_train = np.zeros((len(ids_1) - len(removal), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(ids_1) - len(removal), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
types_train = []

# Set counters and list for saving IDs of images
count = 0
index = 0
ids_train = []

print('Load train images and masks from FOLD 1 and resize')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(ids_1), total=len(ids_1)):
    ids_train.append(id_)
    path = TRAIN_PATH_IMG_1 + '/' + id_
    
    if path not in removal:  # Check if image is not an outlier
        # get numbers from id_
        number = re.findall(r'\d+', id_) 
        res = list(map(int, number))
                
        # Load type        
        type_ = np.load( f'../input/nuclei-detection-for-segmentation/Fold__1/Fold_1/content/Fold 1/types/type_{res[0]}.npy')
        type_ = type_.tolist()
        types_train.append(type_)       

        # Load image
        img = np.load(path)
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_train[index] = img
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
                
        # Load mask
        path = TRAIN_PATH_MASK_1 + '/mask_' + str(res[0]) + ".npy"
        mask_ = np.load(path, allow_pickle=True)
        positive_pixel_count = mask_[:,:,5].sum() # assumes binary mask
        if positive_pixel_count == 0:
            mask_ = mask_[:,:,5]
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)
            mask_ = np.array(mask_, dtype=bool).astype(np.uint8)
            mask = np.maximum(mask, mask_)

            Y_train[index] = mask
        
        else:       

            mask_ = mask_[:,:,5]
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)

            mask_ = np.array(mask_, dtype=bool).astype(np.uint8)
            mask_ = np.where((mask_==0)|(mask_==1), mask_^1, mask_)
            mask = np.maximum(mask, mask_)

            Y_train[index] = mask

        index += 1
    else:
        count += 1
        
print("Finished part 1 TRAIN SET")
print("noise and outliers: ", count)
print("Index: ", index)

### Load the validation data (Fold 2)

In [ ]:
# Load validation images and corresponding masks from Fold 2

removal = outliers_val + noise_val + outliers_val_PCA

# Create empty arays to store images and masks
X_val = np.zeros((len(ids_2) - len(removal), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_val = np.zeros((len(ids_2) - len(removal), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
types_val = []

# Set counters and list for saving IDs of images
count = 0
index = 0
ids_val = []

print('Load validation images and masks from FOLD 2 and resize')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(ids_2), total=len(ids_2)):
    ids_val.append(id_)
    path = VAL_PATH_IMG_2 + '/' + id_
    
    if path not in removal:   # Check if image is not an outlier
        # get numbers from id_
        number = re.findall(r'\d+', id_) 
        res = list(map(int, number))
        
        # Load type        
        type_ = np.load( f'../input/nuclei-detection-for-segmentation/Fold__2/Fold_2/content/Fold 2/types/type_{res[0]}.npy')
        type_ = type_.tolist()
        types_val.append(type_) 
        
        # load image
        img = np.load(path)
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_val[index] = img
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        
        # load mask
        path = VAL_PATH_MASK_2 + '/mask_' + str(res[0]) + ".npy"
        mask_ = np.load(path, allow_pickle=True)
        positive_pixel_count = mask_[:,:,5].sum() # assumes binary mask
        if positive_pixel_count == 0:
            mask_ = mask_[:,:,5]
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)
            mask_ = np.array(mask_, dtype=bool).astype(np.uint8)
            mask = np.maximum(mask, mask_)

            Y_val[index] = mask
        
        else:       

            mask_ = mask_[:,:,5]
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)

            mask_ = np.array(mask_, dtype=bool).astype(np.uint8)
            mask_ = np.where((mask_==0)|(mask_==1), mask_^1, mask_)
            mask = np.maximum(mask, mask_)

            Y_val[index] = mask
        
        index += 1


    else:
        count += 1
        
print("Finished part 2 VAL SET")
print("Index: ", index)
print("noise and outliers: ", count)

### Add image augmentation


In [ ]:
from keras.preprocessing import image

# Creating the training Image and Mask generator
image_datagen = image.ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='reflect')
mask_datagen = image.ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='reflect')

# Keep the same seed for image and mask generators so they fit together

image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(Y_train, augment=True, seed=seed)

x=image_datagen.flow(X_train,batch_size=BATCH_SIZE,shuffle=True, seed=seed)
y=mask_datagen.flow(Y_train,batch_size=BATCH_SIZE,shuffle=True, seed=seed)


# Creating the validation Image and Mask generator
image_datagen_val = image.ImageDataGenerator()
mask_datagen_val = image.ImageDataGenerator()

image_datagen_val.fit(X_val, augment=True, seed=seed)
mask_datagen_val.fit(Y_val, augment=True, seed=seed)

x_val=image_datagen_val.flow(X_val,batch_size=BATCH_SIZE,shuffle=True, seed=seed)
y_val=mask_datagen_val.flow(Y_val,batch_size=BATCH_SIZE,shuffle=True, seed=seed)

train_generator = zip(x, y)
val_generator = zip(x_val, y_val)

### Set evalutation metrics

In [ ]:
smooth = K.epsilon()
threshold = 0.5
label_smoothing = 0.0
bce_weight = 0.5

def iou(y_true, y_pred):
    overlap = tf.math.logical_and((y_true > threshold),(y_pred > threshold))
    union = tf.math.logical_or((y_true > threshold),(y_pred > threshold))
    iou = (tf.cast(tf.math.count_nonzero(overlap),tf.float32) + smooth) / (tf.cast(tf.math.count_nonzero(union),tf.float32) + smooth)
    return iou

def jaccard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)    
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return (intersection +1.0) / (K.sum(y_true) + K.sum(y_pred) - intersection + 1.0)

def jaccard_coef_loss(y_true, y_pred):
    return -jaccard_coef(y_true, y_pred)

def dice_coef(y_true, y_pred, smooth=1):
    #y_pred = y_pred[y_pred > threshold].astype('uint8')
    #y_true = y_true[y_true > threshold].astype('uint8')
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    return K.mean( (2. * intersection + smooth) / (union + smooth))

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

### Implement U-net model

source: https://towardsdatascience.com/nucleus-segmentation-using-u-net-eceb14a9ced4

In [ ]:
import tensorflow_addons as tfa

inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)



### Compile and run model

In [ ]:
model = Model(inputs=[inputs], outputs=[outputs], name="UNET")
# epochs=10
# learning_rate = 0.0001
# decay_rate = learning_rate / epochs
# momentum = 0.8
# adam = Adam(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[iou, jaccard_coef, dice_coef])
model.summary()

# # define the learning rate change 
# def exp_decay(epoch):
#     lrate = learning_rate * np.exp(-decay_rate*epoch)
#     return lrate
    
# # learning schedule callback
# loss_history = History()
# lr_rate = LearningRateScheduler(exp_decay)
# callbacks_list = [loss_history, lr_rate, checkpointer]

#earlystopper = EarlyStopping(patience=3, verbose=1)
checkpointer = ModelCheckpoint('model-UNET-1.h5', verbose=1, save_best_only=True)
results = model.fit_generator(train_generator, validation_data=val_generator, validation_steps=10, steps_per_epoch=250,
                              epochs=40, callbacks=[checkpointer])

### Plot evaluation metrics and loss vs epochs


In [ ]:
print(results.history.keys())

figure, axes = plt.subplots(nrows=3, ncols=1, figsize=(12,7))
axes[0].plot(results.history['loss'])
axes[0].plot(results.history['val_loss'])
axes[0].legend(["train", "val"])
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Jaccard loss')

axes[1].plot(results.history['jaccard_coef'])
axes[1].plot(results.history['val_jaccard_coef'])
axes[1].legend(["train", "val"])
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Jaccard')

axes[2].plot(results.history['dice_coef'])
axes[2].plot(results.history['val_dice_coef'])
axes[2].legend(["train", "val"])
axes[2].set_xlabel('epoch')
axes[2].set_ylabel('F1')

figure.suptitle('Performance metrics in relation to epoch')

### Load the test data

In [ ]:
removal = outliers_test + noise_test + outliers_test_PCA

# Create empty arrays to store images and masks
X_test = np.zeros((len(ids_3) - len(removal), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(ids_3) - len(removal), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
types_test = []

# Set counters and list for saving IDs of images
count_test = 0
index = 0
ids_test = []
sizes_test = []


print('Load test images and masks from FOLD 3 and resize')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(ids_3), total=len(ids_3)):
    
    path = TEST_PATH_IMG_3 + '/' + id_
    
    if path not in removal:  # Check if image is not an outlier
        ids_test.append(id_)
        # get numbers from id_
        number = re.findall(r'\d+', id_) 
        res = list(map(int, number))
        
        # Load type        
        type_ = np.load( f'../input/nuclei-detection-for-segmentation/Fold__3/Fold_3/content/Fold 3/types/type_{res[0]}.npy')
        type_ = type_.tolist()
        types_test.append(type_)       

        # Load image
        img = np.load(path)
        sizes_test.append([img.shape[0], img.shape[1]])
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_test[index] = img
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
                
        # Load mask
        path = TEST_PATH_MASK_3 + '/mask_' + str(res[0]) + ".npy"
        mask_ = np.load(path, allow_pickle=True)
        
        positive_pixel_count = mask_[:,:,5].sum() # assumes binary mask
        if positive_pixel_count == 0:
            mask_ = mask_[:,:,5]
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)
            mask_ = np.array(mask_, dtype=bool).astype(np.uint8)
            mask = np.maximum(mask, mask_)

            Y_test[index] = mask
        
        else:       

            mask_ = mask_[:,:,5]
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)

            mask_ = np.array(mask_, dtype=bool).astype(np.uint8)
            mask_ = np.where((mask_==0)|(mask_==1), mask_^1, mask_)
            mask = np.maximum(mask, mask_)

            Y_test[index] = mask

        index += 1
    else:
        count_test += 1
        
print("Finished TEST SET")
print("Noise and outliers: ", count_test)
print("Index: ", index)

### Create predictions

In [ ]:
custom_obj = {}
#custom_obj['jaccard_coef_loss'] = jaccard_coef_loss
custom_obj['jaccard_coef'] = jaccard_coef
custom_obj['dice_coef'] = dice_coef
custom_obj['dice_coef_loss'] = dice_coef_loss
custom_obj['iou'] = iou

# Define model
model = load_model('../input/unet-diceloss/model-UNET-diceloss.h5', custom_objects = custom_obj)
preds_train = model.predict(X_train, verbose=1)
#preds_val = model.predict(X_val, verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
#preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

### Check robustness of predctions (F1-score and Jaccard score)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

f1_scores = []
jaccard_scores = []
threshold = []
final_f1 = []
final_jaccard = []

#print("Training time: ", end-start)
print("len pred: ",len(preds_train_t))
print("len y_train: ",len(Y_train))

for thres in range(0, 105, 5):
    thres_ = thres/100
    threshold.append(thres_)
    preds_test_t = (preds_test > thres_).astype(np.uint8)
    
    f1_scores = []
    jaccard_scores = []
                    
    for i in range(0, len(Y_test)):

        f1 = f1_score(Y_test[i].reshape(-1), preds_test_t[i].reshape(-1))
        f1_scores.append(f1)
        jacc = jaccard_score(Y_test[i].reshape(-1), preds_test_t[i].reshape(-1))
        jaccard_scores.append(jacc)
    
    final_f1.append(sum(f1_scores)/len(Y_test))
    final_jaccard.append(sum(jaccard_scores)/len(Y_test))

### Processing of the predictions for calculation of PQ and Cell-Counting Accuracy


In [ ]:
before = np.copy(preds_test_t)

for i in range(0, len(preds_test_t)):
    before[i] = np.where(before[i]==1, (Y_test_panoptic[i]*2 + before[i]), before[i])

for i in range(0, len(before)):
    before[i] = np.where(before[i] > 1, before[i] -1 , before[i])

trial = np.copy(before)

shape_ = trial.shape
print(shape_)

for x in range(0, 4):
    print(x)
    for i in range(0, shape_[0]):
        trial[i] = np.rot90(trial[i])

        for row in range(0, shape_[1]):
            for col in range(0, shape_[2]):
                if trial[i][row][col] == 1:
                    if row != 223 and trial[i][row+1][col] not in (1, 0):
                        trial[i][row][col] = trial[i][row+1][col]
                    elif row != 0 and trial[i][row-1][col] not in (1, 0):
                        trial[i][row][col] = trial[i][row-1][col]
                    elif col != 223 and trial[i][row][col+1] not in (1,0):
                        trial[i][row][col] = trial[i][row][col+1]                         
                    elif col != 0 and trial[i][row][col-1] not in (1, 0):
                        trial[i][row][col] = trial[i][row][col-1]
                    
                    elif row not in (223,222) and trial[i][row+2][col] not in (1, 0):
                        trial[i][row][col] = trial[i][row+2][col]
                    elif col not in (222,223) and trial[i][row][col+2] not in (1,0):
                        trial[i][row][col] = trial[i][row][col+2]
                    elif row not in (0,1) and trial[i][row-2][col] not in (1, 0):
                        trial[i][row][col] = trial[i][row-2][col]
                    elif col not in (0,1) and trial[i][row][col-2] not in (1,0):
                        trial[i][row][col] = trial[i][row][col-2]
                        
                    elif row not in (223,222,221) and trial[i][row+3][col] not in (1, 0):
                        trial[i][row][col] = trial[i][row+3][col]                 
                    elif row not in (0,1,2) and trial[i][row-3][col] not in (1, 0):
                        trial[i][row][col] = trial[i][row-3][col]                   
                    elif col not in (222,223,221) and trial[i][row][col+3] not in (1,0):
                        trial[i][row][col] = trial[i][row][col+3]
                    elif col not in (0,1,2) and trial[i][row][col-3] not in (1,0):
                        trial[i][row][col] = trial[i][row][col-3]
                    else:
                        trial[i][row][col] = trial[i][row][col]
                        


### Cell-Counting Accuracy calculation


In [ ]:
# BASED ON: https://stackoverflow.com/questions/58751101/count-number-of-cells-in-the-image
import copy

len_contour_list_true = []
after_thresholds_true = []
area_nuclei = []

example = 70

for i in range(0, len(Y_test)):

    mask = (Y_test_panoptic[i]*2).astype('uint8')    
    img  = np.copy(X_test[i])
    drawing = np.copy(img)
    count_cnts = 0

    IDs = np.unique(mask)
    for ID in IDs:
        if ID != 0:
            mask_per_id = np.copy(mask)
            for x in range(0, len(mask)):                
                mask_per_id[x] = np.where(mask[x] != ID, 0, mask[x])

            cnts = cv2.findContours(mask_per_id, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            
            count_cnts += len(cnts)            
            
            #cv2.drawContours(drawing, cnts, -1, (0, 255, 0), thickness=3)

    len_contour_list_true.append(count_cnts)
    
#     figure, (ax3, ax4) = plt.subplots(1,2, figsize=(10,10))
#     ax1.imshow(mask * 255)
#     print("nr of nuclei true: ", count_cnts)

    # count nuclei area for every image in test set
    for c in cnts:
        area = cv2.contourArea(c)
        area_nuclei.append(area)


#### PREDICTED MASK ####

len_contour_list_pred = []
after_thresholds_pred = []

for i in range(0, len(Y_test)):

    mask_ = trial[i].astype('uint8')    
    img_  = np.copy(X_test[i])
    count_cnts = 0

    IDs = np.unique(mask_)
    for ID in IDs:
        if ID != 0:
            mask_per_id = np.copy(mask_)
            for x in range(0, len(mask_)):                
                mask_per_id[x] = np.where(mask_[x] != ID, 0, mask_[x])

            cnts = cv2.findContours(mask_per_id, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            
            count_cnts += len(cnts)
            
#             cv2.drawContours(drawing, cnts, -1, (255,0,0), thickness=2)
            
    len_contour_list_pred.append(count_cnts)
#     print("nr of nuclei pred: ", count_cnts)


#     ax2.imshow(mask_ * 255)
#     ax4.imshow(drawing)
#     ax4.set_title('U-net (red) vs GT (green)', fontsize=15)
#     ax4.axis('off')
#     ax3.imshow(img_)
#     ax3.set_title('Example image',  fontsize=15)
#     ax3.axis('off')

#     ax2.plot(cnts_)
#     print(cnts_)

abs_diff = []
for i in range(0, len(len_contour_list_true)):
    if len_contour_list_true[i] is not 0:
        abs_diff.append((abs(len_contour_list_true[i] - len_contour_list_pred[i]) / len_contour_list_true[i]))

print("mean of abs diff: ", statistics.mean(abs_diff))
print("max abs diff: ", max(abs_diff))
print("min abs diff: ", min(abs_diff))
print("Cell-counting accuracy: ", 1-statistics.mean(abs_diff))
print("std of abs diff: ", statistics.stdev(abs_diff))

### Calculation of PQ
source: https://www.kaggle.com/ipateam/u-net-with-binary-labels

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

def get_fast_pq(true, pred, match_iou=0.5):
    """
    `match_iou` is the IoU threshold level to determine the pairing between
    GT instances `p` and prediction instances `g`. `p` and `g` is a pair
    if IoU > `match_iou`. However, pair of `p` and `g` must be unique
    (1 prediction instance to 1 GT instance mapping).
    If `match_iou` < 0.5, Munkres assignment (solving minimum weight matching
    in bipartite graphs) is caculated to find the maximal amount of unique pairing.
    If `match_iou` >= 0.5, all IoU(p,g) > 0.5 pairing is proven to be unique and
    the number of pairs is also maximal.
    Fast computation requires instance IDs are in contiguous orderding
    i.e [1, 2, 3, 4] not [2, 3, 6, 10]. Please call `remap_label` beforehand
    and `by_size` flag has no effect on the result.
    Returns:
        [dq, sq, pq]: measurement statistic
        [paired_true, paired_pred, unpaired_true, unpaired_pred]:
                      pairing information to perform measurement
    """
    assert match_iou >= 0.0, "Cant' be negative"
    
    positive_pixel_count = true.sum()
    
    if positive_pixel_count == 0 :
        print('positive = 0')
        return None
    
    
    else:
    
        true = np.copy(true)
        pred = np.copy(pred)
        true_id_list = list(np.unique(true))
        pred_id_list = list(np.unique(pred))

        true_masks = [None, ]
        for t in true_id_list[1:]:
            t_mask = np.array(true == t, np.uint8)
            true_masks.append(t_mask)

        pred_masks = [None, ]
        for p in pred_id_list[1:]:
            p_mask = np.array(pred == p, np.uint8)
            pred_masks.append(p_mask)

        # prefill with value
        pairwise_iou = np.zeros([len(true_id_list) - 1,
                                 len(pred_id_list) - 1], dtype=np.float64)

        # caching pairwise iou
        for true_id in true_id_list[1:]:  # 0-th is background
            t_mask = true_masks[true_id]
            pred_true_overlap = pred[t_mask > 0]
            pred_true_overlap_id = np.unique(pred_true_overlap)
            pred_true_overlap_id = list(pred_true_overlap_id)
            for pred_id in pred_true_overlap_id:
                if pred_id == 0:  # ignore
                    continue  # overlaping background
                p_mask = pred_masks[pred_id]
                total = (t_mask + p_mask).sum()
                inter = (t_mask * p_mask).sum()
                iou = inter / (total - inter)
                pairwise_iou[true_id - 1, pred_id - 1] = iou
        #
        if match_iou >= 0.5:
            paired_iou = pairwise_iou[pairwise_iou > match_iou]
            pairwise_iou[pairwise_iou <= match_iou] = 0.0
            paired_true, paired_pred = np.nonzero(pairwise_iou)
            paired_iou = pairwise_iou[paired_true, paired_pred]
            paired_true += 1  # index is instance id - 1
            paired_pred += 1  # hence return back to original
        else:  # * Exhaustive maximal unique pairing
            #### Munkres pairing with scipy library
            # the algorithm return (row indices, matched column indices)
            # if there is multiple same cost in a row, index of first occurence
            # is return, thus the unique pairing is ensure
            # inverse pair to get high IoU as minimum
            paired_true, paired_pred = linear_sum_assignment(-pairwise_iou)
            ### extract the paired cost and remove invalid pair
            paired_iou = pairwise_iou[paired_true, paired_pred]

            # now select those above threshold level
            # paired with iou = 0.0 i.e no intersection => FP or FN
            print(paired_true)
            paired_true = list(paired_true[paired_iou > match_iou] + 1)
            paired_pred = list(paired_pred[paired_iou > match_iou] + 1)
            paired_iou = paired_iou[paired_iou > match_iou]

        # get the actual FP and FN
        unpaired_true = [idx for idx in true_id_list[1:] if idx not in paired_true]
        unpaired_pred = [idx for idx in pred_id_list[1:] if idx not in paired_pred]
        # print(paired_iou.shape, paired_true.shape, len(unpaired_true), len(unpaired_pred))

        #
        tp = len(paired_true)
        fp = len(unpaired_pred)
        fn = len(unpaired_true)


        # get the F1-score i.e DQ
        dq = tp / (tp + 0.5 * fp + 0.5 * fn)
        #dq = f1_score(true.reshape(-1), pred.reshape(-1))
       
        # get the SQ, no paired has 0 iou so not impact
        sq = paired_iou.sum() / (tp + 1.0e-6)
        

        return [dq, sq, dq * sq], [paired_true, paired_pred, unpaired_true, unpaired_pred]
    
        

### Calculate and plot wrong predicted pixels for example images

In [ ]:
wrong_pixels = np.zeros((len(ids_3) - len(removal), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

for i in range(0, len(wrong_pixels)):
    
    wrong_pixels[i] = Y_test[i] - preds_test_t[i]



In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(20,20))

example1 = 15
axs[0,0].imshow(X_test[example1])
axs[0,0].set_title('Image '+types_train[example1]+'', fontsize=15)
axs[0,1].imshow(Y_test[example1]*255 ,cmap='gray', vmin=0, vmax=255)
axs[0,1].set_title('Ground truth', fontsize=15)
axs[0,2].imshow(preds_test_t[example1]*255,cmap='gray', vmin=0, vmax=255)
axs[0,2].set_title('U-net', fontsize=15)
axs[0,3].imshow(wrong_pixels[example1].astype('uint8')*255, vmin=0, vmax=255)
axs[0,3].set_title('Wrong predicted pixels', fontsize=15)

example2 = 12
axs[1,0].imshow(X_test[example2])
axs[1,0].set_title('Image '+types_train[example2]+'', fontsize=15)
axs[1,1].imshow(Y_test[example2]*255 ,cmap='gray', vmin=0, vmax=255)
axs[1,1].set_title('Ground truth', fontsize=15)
axs[1,2].imshow(preds_test_t[example2]*255,cmap='gray', vmin=0, vmax=255)
axs[1,2].set_title('U-net', fontsize=15)
axs[1,3].imshow(wrong_pixels[example2].astype('uint8')*255, vmin=0, vmax=255)
axs[1,3].set_title('Wrong predicted pixels', fontsize=15)

example3 = 100
axs[2,0].imshow(X_test[example3])
axs[2,0].set_title('Image '+types_train[example3]+'', fontsize=15)
axs[2,1].imshow(Y_test[example3]*255 ,cmap='gray', vmin=0, vmax=255)
axs[2,1].set_title('Ground truth', fontsize=15)
axs[2,2].imshow(preds_test_t[example3]*255,cmap='gray', vmin=0, vmax=255)
axs[2,2].set_title('U-net', fontsize=15)
axs[2,3].imshow(wrong_pixels[example3].astype('uint8')*255, vmin=0, vmax=255)
axs[2,3].set_title('Wrong predicted pixels', fontsize=15)

example4 = 1000
axs[3,0].imshow(X_test[example4])
axs[3,0].set_title('Image '+types_train[example4]+'', fontsize=15)
axs[3,1].imshow(Y_test[example4]*255 ,cmap='gray', vmin=0, vmax=255)
axs[3,1].set_title('Ground truth', fontsize=15)
axs[3,2].imshow(preds_test_t[example4]*255,cmap='gray', vmin=0, vmax=255)
axs[3,2].set_title('U-net', fontsize=15)
axs[3,3].imshow(wrong_pixels[example4].astype('uint8')*255, vmin=0, vmax=255)
axs[3,3].set_title('Wrong predicted pixels', fontsize=15)

fig.savefig('./example_pred_Unet.jpg')